# ELYZA-tasks-100 による性能評価

In [ ]:
!pip install accelerate datasets jsonlines sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.7 MB/s eta 0:00:00


## ELYZA-tasks-100 データセットのダウンロード

モデルのダウンロードには 🤗datasets ライブラリを用いる。

データは `/content/dataset.jsonl` に保存する。

In [ ]:
import json
from pathlib import Path

from datasets import load_dataset

# Load the dataset
ds = load_dataset("elyza/ELYZA-tasks-100")

# Function to convert dataset to JSONL format and print
def dataset_to_jsonl(dataset, filename):
    with open(filename, 'w', encoding='utf-8') as file:
        for entry in dataset:
            # Construct JSON object
            json_obj = {
                "input_text": entry['input'],
                "output_text": entry['output'],
                "eval_aspect": entry['eval_aspect']
            }

            # Write JSON object to file in JSONL format
            json_str = json.dumps(json_obj, ensure_ascii=False)
            file.write(json_str + '\n')

# Convert and write the dataset to a file in JSONL format
path_jsonl = Path("/content/dataset.jsonl")

if (path_jsonl.parent is not None) and (not path_jsonl.parent.exists()):
    path_jsonl.parent.mkdir(parents=True, exist_ok=True)

dataset_to_jsonl(ds["test"], path_jsonl)


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating test split: 0 examples [00:00, ? examples/s]

## 評価

ELYZA-tasks-100 データセットを用いて、指定したモデルの評価を実行する。

### プロンプトについて

モデルに回答を生成させるときのプロンプトは、各モデルのモデルカード (🤗HuggingFace の README ページ) に記載されているプロンプトを使用する。

> 📒 **ノート**
>
> この方法は、質問を直接入力する場合や、全モデルで共通のプロンプトを用いる場合に比べて公平性が劣る。
> しかし、実利用時は各モデルが最もよい性能を発揮するように、各モデルのプロンプトをチューニングすることが想定される。
> モデルカード記載のプロンプトをチューニング済みのプロンプトと見なすことで、実利用時の性能を比較することを狙いとする。

### ハイパーパラメータについて

ハイパーパラメータは 🤗HuggingFace の Code Snippet のとおりとする（公開者が指定したハイパーパラメータを、チューン済みとみなす）。
ただし、長文の回答を要求する設問もあるため、`max_new_tokens` は `1024` に統一する。

In [ ]:
import json
import jsonlines
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, GenerationConfig

# Setup model and tokenizer
model_name = "deepseek-ai/deepseek-llm-7b-chat"
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    #torch_dtype=torch.float16,
    device_map="auto")
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
    device_map="auto",
    # trust_remote_code=True, # ⚠️ Microsoft/Phi-1.5 のみ必要
)
model.generation_config = GenerationConfig.from_pretrained(model_name) # 🐋 Specific to DeepSeek
model.generation_config.pad_token_id = model.generation_config.eos_token_id # 🐋 Specific to DeepSeek

def format_prompt(input_text):
    # ✋プロンプトは、モデルに合わせて調整（HuggingFace のモデルカード参照）
    messages = [{"role": "user", "content": input_text}]
    input_tensor = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt"
    )
    return input_tensor

def generate_text(input_text):
    input_tensor = format_prompt(input_text)

    # Set seed for reproducibility
    seed = 42
    torch.manual_seed(seed)

    # ✋Hparamsは、モデルに合わせて調整（HuggingFace のモデルカード参照）
    tokens = model.generate(
        input_tensor.to(device=model.device),
        max_new_tokens=1024,
        pad_token_id=tokenizer.pad_token_id,
        bos_token_id=tokenizer.bos_token_id,
        eos_token_id=tokenizer.eos_token_id
    )

    # Remove the input tokens from the generated tokens before decoding
    output_tokens = tokens[0][input_tensor.shape[1]:]
    return tokenizer.decode(output_tokens, skip_special_tokens=True)

def process_dataset(input_file, output_file):
    # Read dataset and generate predictions
    with jsonlines.open(input_file) as reader, jsonlines.open(output_file, mode='w') as writer:
        for obj in reader:
            prompt = obj['input_text']
            generated_text = generate_text(prompt)
            print(f"==============================")
            print(f"Q. {prompt}")
            print(f"A. {generated_text}")
            print(f"")
            writer.write({"pred": generated_text})

# Process the dataset
input_dataset = '/content/dataset.jsonl'
output_predictions = '/content/preds.jsonl'
process_dataset(input_dataset, output_predictions)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Q. 仕事の熱意を取り戻すためのアイデアを5つ挙げてください。
A. 1. 目標設定: 仕事の目標を明確にし、月々の進捗を見積もりましょう。目標は、短期の変化でも感じるように設定して、進捗をめざしていきましょう。
2. 新しい学び: 新しい分野に挑戦したり、新たな知識を身に付けることで、仕事の面での熱意を喚起することができます。
3. 仕事のスケジュールの調整: スケジュールを調整して、よりストレスや疲労感を抑えることができる日程を設定しましょう。
4. 趣味や休みに時間割り当て: 興味のあることや休暇に時間を割り当て、興味のあることに時間を投資することで、仕事の熱意を喚わせます。
5. 連絡先を確認し直し: お客様や共事者との連絡先を確認し直し、お客様や共事者の問題点やバグを早期に見つけ、解決することで、仕事の熱意を喚き出すことができます。

Q. クマが海辺に行ってアザラシと友達になり、最終的には家に帰るというプロットの短編小説を書いてください。
A. 海辺で、クマはまっすぐに見つかりました。彼は海を泳いでいた中で、それぞれの海生物が彼に近づき、彼らの生活の中で見たことのない色々なものが見えるようになりました。クマは、その中でも素晴らしいフードを探すうつむいでいた。

そんなクマが、ある日海辺のサザンテールに出会ったのは、とても奇妙なことでした。サザンテールは、彼女のメモリーに残ったのは、彼女がクマを見つけたとき、彼が海原に浮かぶような美しい光を放ち、彼の毛皮はその光を輝かせているということだった。

その時、クマは彼女に会って、彼の旅のことを話しました。彼は海辺で、多くの新しいことを見つけ、そして友達をとっていきました。そして、彼は彼女に、彼が見た景色や物語を聞かせました。彼女は彼の話を聞いて、彼の旅の中で見たことを想像して、彼女はとても楽しい時間を過ごしました。

そして、クマは彼女に、彼が旅したことを終えたときに、家に帰ることを話しました。彼女は彼に、それはとても良かったことに思っていました。彼女は、彼が旅していることを、彼の家族と友達に話し合い、彼が見たことを彼の家族と友達に共有することを願っていました。

そして、クマは彼女と別れ、家に帰ってきました。彼は、彼が見たことを家族と友達に話し合えば、とても楽しい時間を過ごしたと思いました。そして、彼は、また